# Extração Automatizada de Dados Imunohistoquímicos de Laudos de Mama

## Introdução Técnica Detalhada

Este notebook implementa um pipeline completo para extração automatizada de informações estruturadas de laudos imunohistoquímicos de mama, utilizando processamento de linguagem natural (NLP) através de Large Language Models (LLMs). O foco é identificar e extrair marcadores biológicos relevantes para o diagnóstico e classificação molecular do câncer de mama.

### Objetivo Principal

O objetivo principal deste notebook é extrair de forma automatizada e estruturada os seguintes biomarcadores dos laudos imunohistoquímicos:

- **Receptor de Estrógeno**: Status (POSITIVO/NEGATIVO)
- **Receptor de Progesterona**: Status (POSITIVO/NEGATIVO)
- **HER-2**: Status (POSITIVO/INCONCLUSIVO/NEGATIVO) baseado no escore
- **Ki-67**: Percentual de expressão (valor numérico)
- **CK5/6**: Status (POSITIVO/NEGATIVO)

Após a extração, o notebook classifica os casos em subtipos moleculares de câncer de mama (Luminal A, Luminal B, HER-2 Superexpresso, Triplo Negativo), que são fundamentais para definição do prognóstico e tratamento.

### Tecnologias Utilizadas

O notebook utiliza um ecossistema tecnológico diversificado:

- **Frameworks de Processamento de Dados**:
  - **Apache Spark** (PySpark): Para processamento distribuído de dados
  - **Delta Lake**: Para armazenamento transacional em formato tabular
  - **Pandas**: Para manipulações mais específicas de dados em memória

- **Bibliotecas para Processamento de Linguagem Natural**:
  - **OpenAI API**: Interface com modelos de linguagem avançados
  - **LLaMA-4-Maverick**: Modelo de linguagem utilizado (via endpoint Databricks)
  - **RegEx** (re): Para extração baseada em padrões (abordagem heurística)

- **Bibliotecas de Monitoramento e Gestão**:
  - **MLflow**: Para registro de métricas e experimentos
  - **Octoops**: Framework interno para monitoramento e alertas
  - **tqdm**: Para visualização do progresso de processamento em lotes

- **Bibliotecas Auxiliares**:
  - **json**: Para manipulação de dados em formato JSON
  - **ast**: Para avaliação segura de strings Python

### Fluxo de Trabalho/Etapas Principais

O pipeline segue um fluxo sequencial bem definido:

1. **Configuração do Ambiente**:
   - Instalação de dependências (OpenAI, tqdm, etc.)
   - Importação de bibliotecas necessárias
   - Configuração de sessão Spark e tokens de acesso

2. **Extração de Dados**:
   - Definição de consultas SQL para selecionar laudos relevantes
   - Aplicação de filtros (linha de cuidado 'mama', exame 'IHMAMA', presença de 'carcinoma')
   - Carregamento de dados em um DataFrame Spark

3. **Preparação do Processamento**:
   - Definição das funções de prompt para o LLM
   - Configuração das funções de geração em lote
   - Implementação de funções de extração heurística (para validação)

4. **Processamento dos Laudos**:
   - Conversão para DataFrame Pandas para processamento em memória
   - Envio dos laudos em lotes para o modelo LLM
   - Limpeza e estruturação das respostas em formato JSON

5. **Enriquecimento e Classificação**:
   - Expansão das respostas JSON em colunas individuais
   - Aplicação de regras de classificação molecular
   - Análise e validação dos resultados

6. **Persistência e Monitoramento**:
   - Armazenamento dos resultados em tabela Delta
   - Registro de métricas e desempenho (via MLflow)
   - Geração de alertas em caso de falhas (via Sentinel)

### Dados Envolvidos

#### Fontes de Dados
- **Tabela Principal**: `refined.saude_preventiva.pardini_laudos`
  - Contém os laudos brutos de exames realizados
  - Filtrados para `linha_cuidado = 'mama'` e `sigla_exame = 'IHMAMA'`

#### Tabela de Destino
- **Tabela de Saída**: `refined.saude_preventiva.pardini_laudos_mama_imunohistoquimico`
  - Armazena os resultados processados com as informações extraídas
  - Mantém chaves primárias para rastreabilidade (`ficha`, `id_exame`, `id_marca`, `sequencial`)

#### Colunas Relevantes
- **Entrada**:
  - `laudo_tratado`: Texto completo do laudo imunohistoquímico
  - Identificadores: `id_marca`, `id_unidade`, `id_cliente`, `ficha`, etc.
  - Metadados: `dth_pedido`, `dth_resultado`, `linha_cuidado`, `sexo_cliente`

- **Saída** (Extraída do LLM):
  - `receptor_estrogeno`: Status do receptor de estrógeno
  - `receptor_progesterona`: Status do receptor de progesterona
  - `status_her2`: Classificação do HER-2 baseada no escore
  - `ki67_percentual`: Valor percentual de expressão do Ki-67
  - `status_ck5_6`: Status do marcador CK5/6
  - `categoria_final`: Classificação molecular derivada dos marcadores

### Resultados/Saídas Esperadas

O notebook gera os seguintes resultados:

1. **DataFrame Enriquecido** (`df_final_expanded`):
   - Contém todos os dados originais mais as colunas extraídas pelo LLM

2. **DataFrame Classificado** (`df_final_classif`):
   - Adiciona a coluna `categoria_final` com a classificação molecular
   - Categorias possíveis: "Luminal A", "Luminal B", "Luminal com HER2 Positivo", "HER-2 Superexpresso", "Triplo Negativo", "Indefinido"

3. **Tabela Delta Persistente**:
   - Armazena os resultados em formato Delta Lake para consultas futuras
   - Utiliza operações de merge para atualizar registros existentes

4. **Métricas de Validação** (opcional, quando ativado):
   - Comparação entre extração por LLM e extração heurística
   - Taxas de acerto por campo extraído

### Pré-requisitos

Para executar este notebook são necessários:

- **Ambiente Databricks** com:
  - Runtime que suporte PySpark e Delta Lake
  - Acesso configurado ao endpoint do modelo LLaMA-4-Maverick
  - Token de acesso válido (`DATABRICKS_TOKEN`)

- **Bibliotecas Instaladas**:
  - `openai`: Para interação com a API de LLM
  - `tqdm`: Para visualização de progresso
  - `pandarallel`: Para processamento paralelo de pandas
  - `databricks-feature-store`: Para interação com feature store
  - `octoops`: Para monitoramento e alertas

- **Acesso a Dados**:
  - Permissões de leitura na tabela `refined.saude_preventiva.pardini_laudos`
  - Permissões de escrita na tabela `refined.saude_preventiva.pardini_laudos_mama_imunohistoquimico`

### Considerações Importantes/Observações

- **Filtragem de Laudos**: Apenas laudos que contenham os termos 'mama' e 'carcinoma' são processados, conforme especificado na documentação. Casos negativos para câncer são excluídos.

- **Processamento em Lote**: O notebook implementa processamento em lotes (batch_size=100) para gerenciar a carga na API do modelo e evitar limites de taxa.

- **Validação Heurística**: O código inclui funções para extração baseada em regras (expressões regulares) que podem ser usadas para validar os resultados do LLM.

- **Robustez e Tratamento de Erros**:
  - Tentativas múltiplas (até 3) para lidar com falhas de conexão
  - Tratamento de erros ao converter respostas JSON
  - Sistema de alerta via Sentinel para notificar problemas

- **Limitações**:
  - A precisão da extração depende da qualidade e estrutura dos laudos
  - Laudos com formatação muito atípica podem ter resultados subótimos
  - A conversão do valor percentual de Ki-67 pode exigir ajustes para casos especiais

# Extração de dado - Imunohistoquimico
**Extrair os seguintes labels estruturados:**
- Receptor de Estrógeno (categórica): POSITIVO ou NEGATIVO 
  - "RECEPTOR DE ESTRÓGENO - POSITIVO " 
  - "RECEPTOR DE ESTRÓGENO – NEGATIVO" 

- Receptor de Progesterona (categórica): POSITIVO ou NEGATIVO 
  - "RECEPTOR DE PROGESTERONA - POSITIVO" 
  - "RECEPTOR DE PROGESTERONA - NEGATIVO" 

- Status do HER-2 (categórica): Negativo, inconclusivo ou positivo 
  - "HER-2 - ESCORE 0" = Negativo 
  - "HER-2 - ESCORE 1+" = Negativo 
  - "HER-2  -  ESCORE  2+" = Inconclusivo 
  - "HER-2  -  ESCORE  3+" = Positivo 

- Porcentagem do Ki-67 (numérica):  "KI-67 - POSITIVO EM 20% DAS CÉLULAS NEOPLÁSICAS"
  - Deve ser extraído esse número da porcentagem nessa frase 

- Status CK5/6 (categórica): POSITIVO ou NEGATIVO 
  - "CK5/6 - POSITIVO "ou "CK5/6 - NEGATIVO" 
  
+ Para laudos que não possuem carcinoma, ou seja, casos negativos para câncer não devem se processados no LLM.
------------

## Instalação de Dependências

Esta célula instala as bibliotecas Python necessárias para o funcionamento do notebook. Utiliza comandos mágicos do Jupyter (`%pip install`) para instalar pacotes diretamente no ambiente de execução:

1. **openai (-q)**: Cliente Python para interação com a API do OpenAI, usada para fazer chamadas ao modelo de linguagem. A flag `-q` (quiet) reduz a verbosidade da saída.

2. **tqdm (-q)**: Biblioteca para exibição de barras de progresso, útil para acompanhar o processamento em lotes dos laudos.

3. **pandarallel (-q)**: Extensão do pandas para facilitar a paralelização de operações, melhorando a performance em processamentos de DataFrames.

4. **databricks-feature-store (-q)**: Cliente para interagir com o Databricks Feature Store, um repositório centralizado para features de machine learning.

5. **octoops**: Framework interno para gerenciamento de operações como monitoramento, alertas e integração com sistemas externos.

Estas bibliotecas formam a base do ambiente necessário para a execução do pipeline de extração e processamento dos laudos imunohistoquímicos.

In [ ]:
%pip install openai -q
%pip install tqdm -q
%pip install pandarallel -q
%pip install databricks-feature-store -q
%pip install octoops

## Reinicialização do Ambiente Python

Esta célula executa um comando para reiniciar o kernel Python no ambiente Databricks. A função `dbutils.library.restartPython()` reinicia o interpretador Python, garantindo que todas as bibliotecas recém-instaladas na célula anterior estejam disponíveis e devidamente carregadas no ambiente de execução.

Esta etapa é crucial após a instalação de novas bibliotecas, pois algumas delas só estarão completamente funcionais após a reinicialização do ambiente. Essa prática evita erros comuns como "module not found" ou comportamentos inesperados devido à carga parcial de dependências.

In [ ]:
dbutils.library.restartPython()

## Importação de Bibliotecas e Configuração Inicial

Esta célula realiza a importação de todas as bibliotecas necessárias para o funcionamento do pipeline de processamento de laudos imunohistoquímicos. O código é estruturado em blocos lógicos de importações:

### Processamento de Dados e Spark
- **PySpark**: `SparkSession` para gerenciar a sessão Spark
- **Funções SQL**: Importações de `functions as F`, `Window`, `row_number` para manipulação de DataFrames
- **Tipos de dados**: `pyspark.sql.types.*` para definição de schemas

### Manipulação de Texto e Dados
- **json**: Para manipulação de estruturas JSON
- **re**: Para processamento com expressões regulares
- **os, sys**: Para interação com o sistema operacional
- **pandas, numpy**: Para manipulação de DataFrames em memória

### Monitoramento e Logging
- **mlflow**: Para registro de métricas e experimentos
- **tqdm**: Para exibição de barras de progresso
- **warnings**: Para gerenciamento de alertas

### Integração com LLM
- **openai**: Cliente para interação com a API OpenAI
- **time**: Para controle de intervalos entre tentativas de conexão

### Configuração Inicial
- Criação da sessão Spark com nome "LLM_Extractor"
- Configuração do token de acesso ao Databricks para autenticação da API

Esta configuração abrangente estabelece todo o ambiente necessário para as operações de extração, processamento e persistência de dados que serão executadas nas células seguintes.

In [ ]:
from pyspark.sql import SparkSession
import json
import re
import os
import sys
import mlflow
import time
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
from typing import List, Any
import openai
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from databricks.feature_store import FeatureStoreClient


spark = SparkSession.builder.appName("LLM_Extractor").getOrCreate()

DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get() if dbutils.notebook.entry_point.getDbutils().notebook().getContext() is not None else None

## Desativação da Exibição de Rastreamento do MLflow

Esta célula desativa a exibição automática de elementos de rastreamento do MLflow no notebook. A função `mlflow.tracing.disable_notebook_display()` impede que componentes visuais de rastreamento do MLflow sejam automaticamente renderizados na saída das células.

Esta configuração é útil para manter o notebook limpo e focado nos resultados principais da análise, especialmente quando múltiplos experimentos MLflow são registrados durante a execução. Os dados de rastreamento continuam sendo registrados normalmente, apenas não são exibidos automaticamente no notebook.

In [ ]:
mlflow.tracing.disable_notebook_display()

## Configuração de Tabelas e Filtros para Extração de Dados

Esta célula define os parâmetros essenciais para a extração de dados imunohistoquímicos de laudos de mama. Três componentes principais são configurados:

### 1. Definição da Tabela de Destino
```python
table_anatom = "refined.saude_preventiva.pardini_laudos_mama_imunohistoquimico"
```
Esta variável especifica o caminho completo para a tabela Delta onde serão armazenados os resultados da extração.

### 2. Cláusula WHERE para Processamento Incremental
```python
where_clause = f"""
WHERE
    _datestamp > (
        SELECT MAX(anatom._datestamp)
        FROM {table_anatom} anatom
    )
"""
```
Esta cláusula SQL implementa uma estratégia de processamento incremental, garantindo que apenas registros com timestamp mais recente que o último registro processado sejam extraídos. Isso otimiza o processamento ao evitar a reanálise de dados já processados.

### 3. Filtros de Extração Específicos
```python
filtro_extracao = """
    WHERE
        linha_cuidado  = 'mama'
        AND UPPER(flr.sexo_cliente) = 'F'
        AND sigla_exame IN ("IHMAMA")
        AND laudo_tratado RLIKE '(?i)mama' AND laudo_tratado RLIKE '(?i)carcinoma'
"""
```
Este filtro estabelece critérios específicos para seleção dos laudos:
- **Linha de Cuidado**: Restringe aos exames da linha de cuidado 'mama'
- **Sexo do Cliente**: Seleciona apenas pacientes do sexo feminino (`'F'`)
- **Tipo de Exame**: Filtra apenas exames com sigla "IHMAMA" (Imunohistoquímica de mama)
- **Conteúdo do Laudo**: Garante que apenas laudos que contenham os termos 'mama' e 'carcinoma' (usando expressão regular case-insensitive `(?i)`) sejam processados

Essa configuração precisa garante que apenas os laudos relevantes para a análise imunohistoquímica de carcinoma mamário sejam processados, otimizando o uso de recursos computacionais e do modelo de linguagem.

In [ ]:
table_anatom = "refined.saude_preventiva.pardini_laudos_mama_imunohistoquimico" 

where_clause = f"""
WHERE
    _datestamp > (
        SELECT MAX(anatom._datestamp)
        FROM {table_anatom} anatom
    )
"""

 
filtro_extracao = """
    WHERE
        linha_cuidado  = 'mama'
        AND UPPER(flr.sexo_cliente) = 'F'
        AND sigla_exame IN ("IHMAMA")
        AND laudo_tratado RLIKE '(?i)mama' AND laudo_tratado RLIKE '(?i)carcinoma'
"""

## Consulta SQL para Verificação da Tabela de Destino

Esta célula executa uma consulta SQL direta para examinar o conteúdo da tabela de destino onde os resultados serão salvos. O comando `%sql` é uma "magic command" do Jupyter que permite executar SQL nativo diretamente no ambiente Databricks.

```sql
select *
from refined.saude_preventiva.pardini_laudos_mamo_imunohistoquimico
```

Esta consulta retorna todas as colunas e registros da tabela de destino, permitindo:

- Verificar se a tabela já existe no ambiente
- Examinar a estrutura atual dos dados (schema, colunas)
- Confirmar os dados já existentes na tabela
- Validar os resultados após o processamento

Trata-se de uma etapa exploratória importante para entender o contexto dos dados já processados e confirmar que a estrutura está conforme esperado antes de prosseguir com novos processamentos.

In [ ]:
%sql
select *
from refined.saude_preventiva.pardini_laudos_mamo_imunohistoquimico

## Consulta SQL para Verificação das Siglas de Exame Disponíveis

Esta célula executa uma consulta SQL para identificar todas as siglas de exame distintas disponíveis na tabela de laudos Pardini, filtradas para a linha de cuidado 'mama'. O comando utiliza novamente a "magic command" `%sql` para execução direta de SQL no ambiente Databricks.

```sql
select distinct (sigla_exame)
from refined.saude_preventiva.pardini_laudos
where linha_cuidado = 'mama'
```

Esta consulta tem objetivos exploratórios importantes:

1. **Identificar a diversidade de exames**: Permite conhecer todos os tipos de exame disponíveis relacionados à linha de cuidado de mama
2. **Validar os filtros**: Confirma que a sigla "IHMAMA" (usada no filtro de extração) realmente existe na base de dados
3. **Identificar outras oportunidades**: Pode revelar outros tipos de exame que poderiam ser incluídos na análise no futuro

Esta etapa exploratória é fundamental para confirmar que os filtros aplicados na célula anterior estão adequados à realidade dos dados disponíveis.

In [ ]:
%sql
select distinct (sigla_exame)
from refined.saude_preventiva.pardini_laudos
where linha_cuidado = 'mama'

## Extração de Laudos para Processamento

Esta célula realiza a extração principal dos laudos que serão processados pelo modelo de linguagem. Ela constrói e executa uma consulta SQL complexa que:

### 1. Estrutura da Consulta

A consulta utiliza uma Common Table Expression (CTE) chamada `base` para organizar a lógica:

```python
query = f"""
WITH base AS (
    SELECT
        flr.id_marca, flr.id_unidade, flr.id_cliente, /* campos identificadores */
        flr.ficha, flr.id_item, flr.id_subitem, flr.sequencial, flr.id_exame, /* identificadores do exame */
        flr.dth_pedido, flr.dth_resultado, /* datas */
        flr.sigla_exame, flr.laudo_tratado, flr.linha_cuidado, flr.sexo_cliente, flr._datestamp
    FROM refined.saude_preventiva.pardini_laudos flr  
    {where_clause}  
)
SELECT * FROM base {filtro_extracao}
"""
```

### 2. Campos Selecionados

A seleção inclui todos os campos necessários para:
- Identificação do paciente e do exame (`id_marca`, `id_unidade`, `id_cliente`, `ficha`, etc.)
- Informações temporais (`dth_pedido`, `dth_resultado`, `_datestamp`)
- Conteúdo do laudo (`laudo_tratado`) - fonte principal para a extração de informações
- Metadados do exame (`sigla_exame`, `linha_cuidado`)

### 3. Filtragem

A consulta combina duas cláusulas de filtro:
- `where_clause`: Garante processamento incremental, extraindo apenas registros mais recentes que os já processados
- `filtro_extracao`: Aplica os critérios de filtragem específicos (linha de cuidado 'mama', sigla 'IHMAMA', sexo 'F', presença dos termos 'mama' e 'carcinoma')

### 4. Processamento e Visualização

A consulta é executada pelo Spark SQL e os resultados são armazenados no DataFrame `df_spk`, que é exibido imediatamente com a função `display()` para inspeção visual.

Esta etapa é crucial pois define o conjunto de dados que será enviado para processamento pelo modelo de linguagem, garantindo que apenas laudos relevantes sejam incluídos.

In [ ]:
query = f"""
WITH base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem,
        flr.sequencial, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`
    FROM refined.saude_preventiva.pardini_laudos flr  
    {where_clause}  
    
)
SELECT *
FROM base
{filtro_extracao}
"""
df_spk = spark.sql(query)
display(df_spk)

## Consultas SQL Alternativas (Comentadas)

Esta célula contém consultas SQL alternativas que estão comentadas, mas que oferecem abordagens diferentes para extração de dados. Embora não estejam sendo executadas, elas são mantidas como referência para estratégias alternativas de processamento:

### 1. Consulta para Identificar Registros Não Processados

A primeira consulta comentada (`query_append`) identifica registros que existem na tabela base, mas que ainda não foram processados e inseridos na tabela de destino:

```sql
WITH base AS (
    -- Seleciona dados da tabela fonte
),
sem_extracao AS (
    -- Faz um LEFT JOIN para identificar registros que não estão na tabela de destino
    -- WHERE mb.id_unidade IS NULL filtra apenas os registros ausentes
)
```

Esta abordagem é útil para processamentos incrementais quando já existe uma tabela de destino parcialmente populada.

### 2. Consulta para Processamento Completo

A segunda consulta comentada (`query_all_base`) implementa uma estratégia para processar todos os registros relevantes, sem considerar processamentos anteriores:

```sql
WITH base AS (
    -- Seleciona e converte tipos de colunas (CAST)
    -- Filtra por linha de cuidado e sigla de exame
)
```

### 3. Pós-Processamento (Comentado)

As linhas seguintes demonstram etapas adicionais para preparação dos dados:
- Filtragem por texto usando regex: `df_imunohistoquimico.filter(F.col("laudo_tratado").rlike("(?i)mama"))`
- Conversão para minúsculas: `withColumn("laudo_tratado", F.lower(...))`
- Criação de índice sequencial: `withColumn("index", row_number().over(window) - 1)`

Estas consultas alternativas representam diferentes estratégias que podem ser ativadas conforme a necessidade, substituindo ou complementando a abordagem principal implementada na célula anterior.

In [ ]:
# query_append = """
# WITH base AS (
#     SELECT
#         flr.id_unidade,
#         flr.id_cliente, 
#         flr.id_item, 
#         flr.id_subitem, 
#         flr.id_exame, 
#         flr.dth_pedido,
#         flr.laudo_tratado,
#         flr.sigla_exame,
#         flr.linha_cuidado
#         FROM refined.saude_preventiva.pardini_laudos flr
#     WHERE
#         flr.linha_cuidado   = 'mama'
#         flr.sigla_exame IN ("IHMAMA")
# ),
# sem_extracao AS (
#     SELECT
#         b.id_unidade,
#         b.id_cliente,
#         b.id_item,
#         b.id_subitem,
#         b.id_exame,
#         b.dth_pedido,
#         b.sigla_exame,
#         b.laudo_tratado,
#         b.RAW_CARCINOMA,
#         b.HAS_CARCINOMA
#     FROM base b
#     LEFT JOIN refined.saude_preventiva.pardini_laudos_mamo_imunohistoquimico mb
#       ON mb.id_unidade = b.id_unidade
#      AND mb.id_cliente = b.id_cliente
#      AND mb.id_item    = b.id_item
#      AND mb.id_subitem = b.id_subitem
#      AND mb.id_exame   = b.id_exame
#     WHERE mb.id_unidade IS NULL
# )
# SELECT *
# FROM sem_extracao
# """

# query_all_base = """WITH base AS (
#         SELECT
#             flr.id_unidade,
#             flr.id_cliente, 
#             CAST(flr.id_item AS INT) AS id_item, 
#             CAST(flr.id_subitem AS INT) AS id_subitem, 
#             flr.id_exame, 
#             flr.dth_pedido,
#             flr.laudo_tratado,
#             flr.sigla_exame
#         FROM refined.saude_preventiva.pardini_laudos flr
#         WHERE
#         flr.linha_cuidado = 'mama'
#         AND
#         flr.sigla_exame IN ("IHMAMA")
#     )
#     SELECT
#         id_unidade,
#         id_cliente, 
#         id_item, 
#         id_subitem, 
#         id_exame, 
#         dth_pedido,
#         sigla_exame,
#         laudo_tratado
#     FROM base """

# df_imunohistoquimico = spark.sql(query_all_base)
# df_imunohistoquimico = df_imunohistoquimico.filter(F.col("laudo_tratado").rlike("(?i)mama"))
# df_imunohistoquimico = df_imunohistoquimico.withColumn("laudo_tratado", F.lower(df_imunohistoquimico["laudo_tratado"]))
# window = Window.orderBy(F.monotonically_increasing_id())
# df_imunohistoquimico = df_imunohistoquimico.withColumn("index", row_number().over(window) - 1)

## Contagem de Registros Extraídos

Esta célula executa uma operação simples porém importante: contar o número total de registros obtidos na consulta anterior.

```python
df_spk.count()
```

Esta operação aciona uma ação de processamento no DataFrame Spark (`df_spk`) e retorna um valor numérico representando o número total de linhas no conjunto de dados.

O resultado dessa contagem é fundamental para:

1. **Validação da Extração**: Confirmar se existem dados para processamento (se o resultado for zero, não há registros a serem processados)
2. **Dimensionamento do Processamento**: Entender o volume de dados que serão enviados ao modelo de linguagem
3. **Controle de Fluxo**: Decisões condicionais posteriores podem depender da existência de registros (conforme visto mais adiante no código)

Esta verificação rápida permite avaliar imediatamente se os filtros aplicados na consulta SQL estão funcionando como esperado e se há dados disponíveis para a continuidade do processamento.

In [ ]:
df_spk.count()

## Definição de Funções para Geração de Prompts e Interação com o LLM

Esta célula implementa um conjunto de funções essenciais que orquestram a interação com o modelo de linguagem para extrair informações dos laudos imunohistoquímicos. Três funções principais são definidas:

### 1. Função de Geração do Prompt (`prompt_laudo`)

```python
def prompt_laudo(laudo_texto: str) -> str:
    # Gera o texto do prompt com instruções e o laudo
```

Esta função:
- Recebe o texto do laudo como entrada
- Cria um prompt estruturado com instruções específicas para o modelo
- Inclui detalhes sobre os campos a serem extraídos e suas regras
- Define o formato esperado da resposta (dicionário Python)

O prompt possui:
- **Contexto**: Informação que o texto é um laudo médico de mamografia
- **Instruções de Fallback**: Orientação para retornar "NÃO INFORMADO" quando informações não estiverem presentes
- **Critérios de Extração**: Regras específicas para cada campo:
  - Receptor de Estrógeno: POSITIVO/NEGATIVO/NÃO INFORMADO
  - Receptor de Progesterona: POSITIVO/NEGATIVO/NÃO INFORMADO
  - Status do HER-2: NEGATIVO/INCONCLUSIVO/POSITIVO baseado no score
  - Ki-67 (%): Valor numérico da porcentagem
  - Status CK5/6: POSITIVO/NEGATIVO/NÃO INFORMADO
- **Formato de Saída**: Template de dicionário Python a ser preenchido

### 2. Função de Geração de Resposta (`generate`)

```python
def generate(descricao_agente:str, laudo:str, llm_client) -> str:
    # Configura e envia a requisição para o modelo de linguagem
```

Esta função:
- Recebe três parâmetros: descrição do agente (system prompt), texto do laudo e cliente da API
- Configura o prompt usando a função `prompt_laudo`
- Estrutura as mensagens no formato esperado pela API (system + user)
- Define parâmetros do modelo: "databricks-llama-4-maverick", temperatura 0 (determinístico), tokens máximos, etc.
- Implementa um mecanismo de tentativas (até 3) para lidar com falhas de conexão
- Retorna o texto da resposta do modelo

### 3. Função de Processamento em Lote (`batch_generate`)

```python
def batch_generate(descricao_agente, laudos, llm_client, batch_size=25):
    # Processa múltiplos laudos em lotes
```

Esta função:
- Recebe uma lista de laudos e processa em lotes de tamanho definido (default: 25)
- Reinicializa o cliente OpenAI com o token Databricks e URL do endpoint
- Usa `tqdm` para exibir barras de progresso durante o processamento
- Para cada laudo no lote, chama a função `generate` e armazena as respostas
- Retorna uma lista com todas as respostas do modelo

### 4. Função de Limpeza e Conversão (`limpar_e_converter`)

```python
def limpar_e_converter(item):
    # Limpa e converte a resposta do modelo para um dicionário Python
```

Esta função:
- Recebe a resposta textual do modelo (geralmente contendo um bloco de código JSON/Python)
- Remove marcadores de código (```python, ```) usando expressões regulares
- Converte o texto limpo para um objeto Python (dicionário)
- Implementa tratamento de erros, retornando um dicionário padrão com "NÃO INFORMADO" em caso de falha na conversão

Estas funções formam o núcleo do sistema de extração de informações, encapsulando a lógica de interação com o modelo de linguagem, formatação de prompts, processamento em lote e tratamento de respostas.

In [ ]:
def prompt_laudo(laudo_texto: str) -> str:
    prompt = f"""A seguir está um laudo médico de mamografia, conforme indicado abaixo. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

    Laudo clínico:
    \"\"\"{laudo_texto}\"\"\"

    ### Critérios de extração:

    - **Receptor de Estrogênio**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Receptor de Progesterona**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Status do HER-2**: retorne se o Status do HER-2 é "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO". Com base no score seguindo as regras:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"

    - **Ki-67 (%)**: retorne o valor numérico da porcentagem de positividade do KI-67.

    - **Status do CK5/6**: retorne "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO" do Status do CK5/6.

    ### Saída esperada (dicionário Python válido):
    ```python
    {{
    "receptor_estrogeno": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "receptor_progesterona": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "status_her2": "POSITIVO" | "POSITIVO" | "INCONCLUSIVO" |  "NÃO INFORMADO",
    "ki67_percentual": float |  0,
    "status_ck5_6": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO"
    }}
    ```
    """
    return prompt.strip()

def generate(descricao_agente:str, laudo:str, llm_client) -> str:
    """
    Gera o resultado da análise de um laudo
    Params:
        descricao_agente: descricao do agente que a LLM representa (primeira mensagem enviada à LLM)
        prompt: prompt base que será utilizado para gerar a análise
        laudo: laudo a ser analisado (incluido dentro do prompt)
        llm_client: cliente da API da LLM
    Return:
        response_message: resposta da LLM
    """
    prompt = prompt_laudo(laudo)
    messages = [
        # Utilizamos o primeiro prompt para contextualizar o llm do que ele deve fazer. 
        # No exemplo utilizamos a abordagem Role, Task, Output, Prompting.
        # Mas sintam-se a vontade para alterar de acordo com a necessidade
        {
            "role": "system",
            "content": descricao_agente
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    model_params = {
        "model": "databricks-llama-4-maverick",
        "messages": messages,
        "temperature": 0,
        "max_tokens": 4000,
        "top_p": 0.75,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
    connection_retry = 0
    while connection_retry < 3:
        try:
            response = llm_client.chat.completions.create(**model_params)
            response_message = response.choices[0].message.content
            break
        # TODO: verificar se a excessao é de conexao
        except (ConnectionError, TimeoutError) as e:
            connection_retry += 1
            print("Sem reposta do modelo")
            print(str(e))
            print("Tentando novamente...")
            time.sleep(0.1)
        except Exception as e:
            raise e

    if connection_retry >= 3:
        response_message = ''
    
    return response_message


def batch_generate(descricao_agente, laudos, llm_client, batch_size=25):
    responses = []
    
    llm_client = openai.OpenAI(
        api_key=DATABRICKS_TOKEN,
        base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
    )
    
    # Dividir em lotes
    for i in range(0, len(laudos), batch_size):
        laudos_batch = laudos[i:i+batch_size]
        for laudo in tqdm(laudos_batch, desc=f"Processando lote {i//batch_size + 1}", total=len(laudos_batch)):
            responses.append(generate(descricao_agente, laudo, llm_client))
    
    return responses

def limpar_e_converter(item):
    try:
        item_limpo = re.sub(r"```(?:python)?", "", item).replace("```", "").strip()
        return json.loads(item_limpo)
    except Exception as e:
        print(f"Erro ao converter resposta: {e}")
        return {
            "receptor_estrogeno": "NÃO INFORMADO",
            "receptor_progesterona": "NÃO INFORMADO",
            "status_her2": "NÃO INFORMADO",
            "ki67_percentual": "NÃO INFORMADO",
            "status_ck5_6": "NÃO INFORMADO"
        }

## Implementação de Funções de Extração Heurística

Esta célula implementa um conjunto de funções baseadas em regras (expressões regulares) para extração de informações dos laudos, criando um mecanismo "pseudo-gold standard" que pode ser usado para avaliar e comparar os resultados do modelo de linguagem. O código está organizado em várias funções especializadas:

### 1. Funções Específicas de Extração

Cinco funções extraem informações específicas dos laudos usando expressões regulares:

#### a) `extrai_receptor_estrogeno(txt)`
- Busca termos como "receptor de estrogenio: positivo" 
- Busca abreviações como "ER+" ou "ER-"
- Retorna "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO"

#### b) `extrai_receptor_progesterona(txt)`
- Similar à função anterior, mas para receptor de progesterona
- Busca padrões como "receptor de progesterona: positivo" 
- Busca abreviações como "PR+" ou "PR-"
- Retorna "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO"

#### c) `extrai_status_her2(txt)`
- Implementa lógica complexa para interpretar escores HER-2
- Identifica padrões como "her-2 escore X+" ou "her2 X+"
- Classifica baseado no escore: 0/1+ como "NEGATIVO", 2+ como "INCONCLUSIVO", 3+ como "POSITIVO"
- Retorna "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO"

#### d) `extrai_ki67_percentual(txt)`
- Extrai o valor numérico da porcentagem de Ki-67
- Busca padrões como "ki-67: 20%" ou "ki 67 15 %"
- Converte o valor encontrado para float
- Retorna um valor float ou 0.0 se não encontrado

#### e) `extrai_status_ck5_6(txt)`
- Busca padrões como "ck5/6: positivo" ou "ck5/6: negativo"
- Busca abreviações como "CK5/6+" ou "CK5/6-"
- Retorna "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO"

### 2. Função de Avaliação Integrada

A função `avalia_extracao_sem_ground_truth_novo(laudo_texto, json_modelo)` é o coração da avaliação:

- **Geração de Gold Standard**: Aplica as funções de extração ao texto do laudo para criar um dicionário `json_heu` com os resultados da extração baseada em regras
- **Preparação do Modelo**: Verifica se `json_modelo` é um dicionário válido, tratando casos onde não é
- **Comparação Campo a Campo**: Para cada campo (receptores, HER-2, CK5/6), compara os valores extraídos heuristicamente com os valores do modelo
- **Tratamento Especial para Ki-67**: Converte o valor do modelo para float para comparação adequada
- **Resultado da Avaliação**: Retorna dois elementos:
  1. `json_heu`: Dicionário com os valores extraídos heuristicamente
  2. `comparacoes`: Dicionário detalhado de comparações para cada campo, incluindo valores extraídos por ambos os métodos e flag de acerto

Esta abordagem de avaliação é valiosa porque permite validar o desempenho do modelo de linguagem mesmo sem um conjunto de dados rotulado manualmente, usando regras heurísticas como referência.

In [ ]:
import ast
#import ace_tools as tools

# ==============================================
# Funções de extração heurística (pseudo-gold) para o novo prompt
# ==============================================

def extrai_receptor_estrogeno(txt: str) -> str:
    """
    Extrai o status de Receptor de Estrogênio: valores possíveis
    "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """
    txt_lower = txt.lower()
    # Padrões comuns: "receptor de estrogênio: positivo" ou "er+: positivo" etc.
    if re.search(r"receptor\s+de\s+estrog[eê]genio\s*[:\-]?\s*positivo", txt_lower):
        return "POSITIVO"
    if re.search(r"receptor\s+de\s+estrog[eê]genio\s*[:\-]?\s*negativo", txt_lower):
        return "NEGATIVO"
    # Abreviações: "ER+" / "ER-"
    if re.search(r"\ber\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\ber\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"
    return "NÃO INFORMADO"

def extrai_receptor_progesterona(txt: str) -> str:
    """
    Extrai o status de Receptor de Progesterona: valores possíveis
    "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """
    txt_lower = txt.lower()
    if re.search(r"receptor\s+de\s+progesterona\s*[:\-]?\s*positivo", txt_lower):
        return "POSITIVO"
    if re.search(r"receptor\s+de\s+progesterona\s*[:\-]?\s*negativo", txt_lower):
        return "NEGATIVO"
    # Abreviações: "PR+" / "PR-"
    if re.search(r"\bpr\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bpr\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"
    return "NÃO INFORMADO"

def extrai_status_her2(txt: str) -> str:
    """
    Extrai o Status do HER-2 baseado em termos de score:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"
    """
    txt_lower = txt.lower()
    # Primeiro, busca padrão "her-2 ... escore X+"
    m = re.search(r"her[-\s]?2.*?escore\s*[:\-]?\s*([0-3]\+?)", txt_lower, flags=re.IGNORECASE)
    if m:
        score = m.group(1)
        if score.startswith("0") or score == "1+":
            return "NEGATIVO"
        if score == "2+":
            return "INCONCLUSIVO"
        if score == "3+":
            return "POSITIVO"
    # Outra forma: "her2 3+" isolado
    m2 = re.search(r"\bher[-]?2\s*[:\-]?\s*([0-3]\+)\b", txt_lower, flags=re.IGNORECASE)
    if m2:
        score2 = m2.group(1)
        if score2 == "1+":
            return "NEGATIVO"
        if score2 == "2+":
            return "INCONCLUSIVO"
        if score2 == "3+":
            return "POSITIVO"
    return "NÃO INFORMADO"

def extrai_ki67_percentual(txt: str) -> float:
    """
    Extrai o valor de Ki-67 em porcentagem. 
    Exemplo no texto: "Ki-67: 20%", "Ki 67 15 %", etc.
    Se não encontrar, retorna 0.0.
    """
    m = re.search(r"ki[-\s]?67.*?[:\-]?\s*(\d{1,3}(?:\.\d+)?)\s*%", txt, flags=re.IGNORECASE)
    if m:
        try:
            return float(m.group(1))
        except ValueError:
            return 0.0
    return 0.0

def extrai_status_ck5_6(txt: str) -> str:
    """
    Extrai o status do CK5/6: "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """
    txt_lower = txt.lower()
    if re.search(r"ck5\s*\/\s*6.*?[:\-]?\s*positivo", txt_lower):
        return "POSITIVO"
    if re.search(r"ck5\s*\/\s*6.*?[:\-]?\s*negativo", txt_lower):
        return "NEGATIVO"
    # Abreviações: "CK5/6+" ou "CK5/6-"
    if re.search(r"\bck5\s*\/\s*6\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bck5\s*\/\s*6\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"
    return "NÃO INFORMADO"

# ==============================================
# Função de avaliação sem ground truth completo (novo conjunto de campos)
# ==============================================

def avalia_extracao_sem_ground_truth_novo(laudo_texto: str, json_modelo: dict):
    """
    Gera pseudo-gold (json_heu) para os campos do novo prompt
    e compara com json_modelo (saída da IA).
    Retorna:
      - json_heu: dicionário com valores heurísticos
      - comparacoes: dicionário que, para cada campo, traz:
          * valor_heu
          * valor_mod
          * acertou (boolean)
    """
    # 1. Gera pseudo-gold (json_heu)
    rei = extrai_receptor_estrogeno(laudo_texto)
    rpr = extrai_receptor_progesterona(laudo_texto)
    her = extrai_status_her2(laudo_texto)
    ki6 = extrai_ki67_percentual(laudo_texto)
    ck6 = extrai_status_ck5_6(laudo_texto)

    json_heu = {
        "receptor_estrogeno": rei,
        "receptor_progesterona": rpr,
        "status_her2": her,
        "ki67_percentual": ki6,
        "status_ck5_6": ck6
    }

    # 2. Prepara json_modelo: se não for dict, converte para dict vazio
    if not isinstance(json_modelo, dict):
        json_modelo = {}

    # 3. Comparações campo a campo
    comparacoes = {}

    # Campos categóricos (strings)
    for campo in ["receptor_estrogeno", "receptor_progesterona", "status_her2", "status_ck5_6"]:
        val_heu = json_heu[campo]
        val_mod = json_modelo.get(campo, "NÃO INFORMADO")
        comparacoes[campo] = {
            "valor_heu": val_heu,
            "valor_mod": val_mod,
            "acertou": (val_heu == val_mod)
        }

    # Campo Ki-67 (%)
    val_heu_ki = json_heu["ki67_percentual"]
    try:
        val_mod_ki = float(json_modelo.get("ki67_percentual", 0))
    except (ValueError, TypeError):
        val_mod_ki = 0.0
    comparacoes["ki67_percentual"] = {
        "valor_heu": val_heu_ki,
        "valor_mod": val_mod_ki,
        "acertou": (val_heu_ki == val_mod_ki)
    }

    return json_heu, comparacoes

## Função de Parse para Strings JSON

Esta célula define uma função auxiliar `parse_json_string()` que proporciona uma maneira segura de converter strings contendo representações de dicionários Python para objetos dicionários reais.

```python
def parse_json_string(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except Exception:
            return {}
    return s
```

A função opera da seguinte forma:

1. **Verificação de Tipo**: Primeiro verifica se a entrada `s` é do tipo string, garantindo que a conversão só seja tentada em objetos apropriados
  
2. **Conversão Segura**: Usa `ast.literal_eval()` em vez de `eval()` para avaliar a string como uma expressão literal Python. Esta é uma abordagem muito mais segura que `eval()`, pois:
   - Só avalia expressões literais de Python (não executa código arbitrário)
   - Aceita apenas tipos básicos como dicionários, listas, strings, números, etc.
   - Previne a execução de código malicioso ou não intencional

3. **Tratamento de Erros**: Captura quaisquer exceções durante a avaliação e retorna um dicionário vazio `{}` em caso de falha, garantindo que um objeto válido sempre será retornado

4. **Passagem Direta**: Se a entrada não for uma string, simplesmente retorna a própria entrada sem modificação

Esta função é especialmente útil para processar as respostas do modelo de linguagem que foram armazenadas como strings mas representam estruturas de dados JSON. Ela garante que o processamento continue mesmo quando há problemas de formatação nas respostas.

In [ ]:
def parse_json_string(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except Exception:
            return {}
    return s

## Função para Agregação de Resultados de Avaliação

Esta célula implementa uma função flexível para agregar resultados de avaliação entre múltiplos laudos, fornecendo métricas de desempenho para cada campo extraído.

```python
def agrega_resultados_dinamico(lista_comparacoes):
    """
    Agraga resultados de uma lista de dicionários de comparações, retornando para cada campo:
      - acertos: número de vezes que 'acertou' == True
      - total: número total de laudos (len(lista_comparacoes))
      - taxa_acerto: acertos / total (ou 0.0 se total == 0)
    """
```

### Componentes Principais:

1. **Estrutura de Entrada**:
   - `lista_comparacoes`: Uma lista de dicionários, onde cada dicionário contém os resultados de comparação para um laudo
   - Cada comparação tem campos como "receptor_estrogeno", "status_her2", etc., cada um com seu próprio dicionário contendo a chave "acertou"

2. **Contagem de Acertos**:
   ```python
   total_laudos = len(lista_comparacoes)
   acertos_por_campo = Counter()
   
   for comp in lista_comparacoes:
       for campo, info in comp.items():
           if info.get("acertou", False):
               acertos_por_campo[campo] += 1
   ```
   - Utiliza `Counter` da biblioteca `collections` para contabilizar eficientemente os acertos
   - Percorre cada comparação e cada campo, incrementando o contador quando "acertou" é verdadeiro

3. **Geração de Métricas**:
   ```python
   resultado = {}
   for campo, acertos in acertos_por_campo.items():
       resultado[campo] = {
           "acertos": acertos,
           "total": total_laudos,
           "taxa_acerto": (acertos / total_laudos) if total_laudos > 0 else 0.0
       }
   ```
   - Para cada campo, calcula três métricas:
     - `acertos`: Total de acertos para o campo específico
     - `total`: Número total de laudos avaliados
     - `taxa_acerto`: Proporção de acertos (com proteção contra divisão por zero)

4. **Tratamento de Campos Ausentes**:
   ```python
   if lista_comparacoes:
       primeira = lista_comparacoes[0]
       for campo in primeira.keys():
           if campo not in resultado:
               resultado[campo] = {"acertos": 0, "total": total_laudos, "taxa_acerto": 0.0}
   ```
   - Garante que todos os campos presentes nas comparações estejam representados nos resultados
   - Inicializa campos não encontrados com zero acertos

Esta função é essencial para a análise quantitativa do desempenho do sistema de extração, permitindo avaliar a precisão para cada tipo de informação extraída dos laudos.

In [ ]:
from collections import Counter

def agrega_resultados_dinamico(lista_comparacoes):
    """
    Agraga resultados de uma lista de dicionários de comparações, retornando para cada campo:
      - acertos: número de vezes que 'acertou' == True
      - total: número total de laudos (len(lista_comparacoes))
      - taxa_acerto: acertos / total (ou 0.0 se total == 0)
    
    Suporta qualquer conjunto de chaves em cada dict, desde que cada valor seja outro dict contendo a chave 'acertou'.
    """
    total_laudos = len(lista_comparacoes)
    acertos_por_campo = Counter()

    # Para cada comparação, percorremos todas as chaves e contamos os acertos
    for comp in lista_comparacoes:
        for campo, info in comp.items():
            # Supondo que info seja um dict com a chave "acertou"
            if info.get("acertou", False):
                acertos_por_campo[campo] += 1

    # Monta o dicionário de saída
    resultado = {}
    for campo, acertos in acertos_por_campo.items():
        resultado[campo] = {
            "acertos": acertos,
            "total": total_laudos,
            "taxa_acerto": (acertos / total_laudos) if total_laudos > 0 else 0.0
        }

    # É possível que exista algum campo em alguma comparação que nunca teve 'acertou' == True.
    # Se quisermos incluir também esses campos (com acertos = 0), podemos varrer as chaves da primeira entrada:
    if lista_comparacoes:
        primeira = lista_comparacoes[0]
        for campo in primeira.keys():
            if campo not in resultado:
                resultado[campo] = {
                    "acertos": 0,
                    "total": total_laudos,
                    "taxa_acerto": 0.0
                }

    return resultado

## Processamento Principal: Extração e Classificação de Biomarcadores

Esta célula contém o bloco principal de execução do notebook, orquestrando o processo completo de extração de informações dos laudos imunohistoquímicos usando o modelo de linguagem. O código está estruturado como um condicional que só executa se existirem registros a serem processados.

### Fluxo de Execução

O processamento ocorre nas seguintes etapas:

1. **Verificação da Disponibilidade de Dados**:
   ```python
   if df_spk.count() > 0:
   ```
   Só prossegue com o processamento se houver registros no DataFrame `df_spk`.

2. **Inicialização do Cliente LLM**:
   ```python
   llm_client = openai.OpenAI(api_key=DATABRICKS_TOKEN,
                           base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints")
   ```
   Configura o cliente para acessar o endpoint do modelo no Databricks.

3. **Definição do Sistema Prompt**:
   ```python
   descricao_agente = "Atue como um médico oncologista especialista em laudos de mamografia."
   ```
   Estabelece o contexto para o modelo, instruindo-o a adotar a persona de um especialista médico.

4. **Preparação dos Dados**:
   ```python
   df_local = df_spk.select("ficha","id_exame","id_marca","sequencial","laudo_tratado").toPandas()
   ```
   Converte um subconjunto do DataFrame Spark para Pandas para facilitar o processamento.

5. **Processamento com o Modelo LLM**:
   ```python
   respostas = batch_generate(descricao_agente, df_local["laudo_tratado"].tolist(), llm_client, batch_size=100)
   respostas_limpa = [limpar_e_converter(item) for item in respostas]
   ```
   Envia os laudos para o modelo em lotes de 100, processa as respostas e converte para dicionários Python.

6. **Integração de Resultados**:
   ```python
   df_local["resposta_llm"] = respostas_limpa
   df_respostas = spark.createDataFrame(df_local)
   ```
   Adiciona as respostas ao DataFrame e converte de volta para Spark.

7. **Junção com Dados Originais**:
   ```python
   df_final = df_spk.join(df_respostas.select("ficha","id_exame","id_marca","sequencial","resposta_llm"), 
                          on=["ficha","id_exame","id_marca","sequencial"], how="inner")
   ```
   Combina as respostas com todos os campos dos dados originais.

8. **Expansão das Respostas em Colunas**:
   ```python
   df_final_expanded = df_final_expanded.select(
       "*",
       col("resposta_struct.receptor_estrogeno").alias("receptor_estrogeno"),
       # ... outras colunas ...
   ).drop("resposta_struct")
   ```
   Extrai cada campo do JSON de resposta como uma coluna individual.

9. **Classificação Molecular**:
   ```python
   df_final_classif = df_final_expanded.withColumn(
       "categoria_final",
       F.when(
           (
               ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
               (F.col("status_her2") == "NEGATIVO") &
               (F.col("ki67_percentual") < 14)
           ),
           "Luminal A"
       )
       # ... outras categorias ...
   )
   ```
   Aplica regras de classificação molecular baseadas nos biomarcadores, categorizando cada caso como:
   - **Luminal A**: RE+ ou RP+, HER2-, Ki-67 < 14%
   - **Luminal B**: RE+ ou RP+, HER2-, Ki-67 >= 14%
   - **Luminal com HER2 Positivo**: RE+ ou RP+, HER2+
   - **HER-2 Superexpresso**: RE-, RP-, HER2+
   - **Triplo Negativo**: RE-, RP-, HER2-

10. **Visualização dos Resultados**:
    ```python
    display(df_final_classif)
    ```
    Exibe o DataFrame final com todas as informações extraídas e classificações.

### Métricas e Avaliação (Comentadas)

O código também inclui seções comentadas para:
- Cálculo de métricas de validação
- Comparação com extração heurística
- Registro de métricas via MLflow

Esta célula representa o núcleo funcional do notebook, transformando os laudos textuais não estruturados em informações clinicamente relevantes e estruturadas.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, IntegerType
from pyspark.sql.functions import from_json, col
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import StringType
import pandas as pd



if df_spk.count() > 0:
    llm_client = openai.OpenAI(api_key=DATABRICKS_TOKEN,
                           base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
                           )
    descricao_agente = "Atue como um médico oncologista especialista em laudos de mamografia."

    # Coleta os dados 
    df_local = df_spk.select("ficha","id_exame","id_marca","sequencial","laudo_tratado").toPandas()

    # Aplica o LLM 
    respostas = batch_generate(descricao_agente, df_local["laudo_tratado"].tolist(), llm_client, batch_size=100)
    respostas_limpa = [limpar_e_converter(item) for item in respostas]

    # Adiciona as respostas ao DataFrame
    df_local["resposta_llm"] = respostas_limpa

    # Converte de volta para Spark
    df_respostas = spark.createDataFrame(df_local)

    # Faz join com o DataFrame original para manter todas as colunas
    df_final = df_spk.join(df_respostas.select("ficha","id_exame","id_marca","sequencial","resposta_llm"), on=["ficha","id_exame","id_marca","sequencial"], how="inner")

    # Definir estrutura
    schema_resposta = StructType([
    StructField("receptor_estrogeno", StringType(), True),
    StructField("receptor_progesterona", StringType(), True),
    StructField("status_her2", StringType(), True),
    StructField("ki67_percentual", DoubleType(), True),
    StructField("status_ck5_6", DoubleType(), True),
    ])

    df_final_expanded = df_final.withColumn("resposta_struct", col("resposta_llm"))

    # expandir resultado llma para colunas

    df_final_expanded = df_final_expanded.select(
    "*",
    col("resposta_struct.receptor_estrogeno").alias("receptor_estrogeno"),
    col("resposta_struct.receptor_progesterona").alias("receptor_progesterona"),
    col("resposta_struct.status_her2").alias("status_her2"),
    col("resposta_struct.ki67_percentual").alias("ki67_percentual"),
    col("resposta_struct.status_ck5_6").alias("status_ck5_6"),
    ).drop("resposta_struct")

    #display(df_final_expanded)

    # a partir dos dados extraídos definir uma classificação final
    df_final_classif = df_final_expanded.withColumn(
            "categoria_final",
            F.when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "NEGATIVO") &
                    (F.col("ki67_percentual") < 14)
                ),
                "Luminal A"
            ).when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "NEGATIVO") &
                    (F.col("ki67_percentual") >= 14)
                ),
                "Luminal B"
            ).when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "POSITIVO")
                ),
                "Luminal com HER2 Positivo"
            ).when(
                (
                    (F.col("receptor_estrogeno") == "NEGATIVO") &
                    (F.col("receptor_progesterona") == "NEGATIVO") &
                    (F.col("status_her2") == "POSITIVO")
                ),
                "HER-2 Superexpresso"
            ).when(
                (
                    (F.col("receptor_estrogeno") == "NEGATIVO") &
                    (F.col("receptor_progesterona") == "NEGATIVO") &
                    (F.col("status_her2") == "NEGATIVO")
                ),
                "Triplo Negativo"
            ).otherwise("Indefinido")
        )

    display(df_final_classif)








    # ###################### Apenas para testes #################
    # df_imunohistoquimico = df_imunohistoquimico.limit(100)
    # ###########################################################

    # rows = df_imunohistoquimico.select("laudo_tratado").collect()
    # laudos = [row.laudo_tratado for row in rows]

    # respostas = batch_generate(descricao_agente, laudos, llm_client, batch_size=10)

    # lista_dicts = [limpar_e_converter(item) for item in respostas]

    # schema = StructType([
    #     StructField("receptor_estrogeno", StringType(), True),
    #     StructField("receptor_progesterona", StringType(), True),
    #     StructField("status_her2", StringType(), True),
    #     StructField("ki67_percentual", StringType(), True),
    #     StructField("status_ck5_6", StringType(), True),
    # ])

    # df_lista = spark.createDataFrame(lista_dicts, schema=schema)

    # w = Window.orderBy(F.lit(1))

    # df_imuno_indexed = (
    #     df_imunohistoquimico
    #     .withColumn("row_id", F.row_number().over(w) - 1)  # subtrai 1 para ficar zero‐based
    # )

    # df_lista_indexed = (
    #     df_lista
    #     .withColumn("row_id", F.row_number().over(w) - 1)
    # )

    # df_final = df_imuno_indexed.join(df_lista_indexed, on="row_id").drop("row_id")

    # df_final = df_final.withColumn(
    #     "categoria_final",
    #     F.when(
    #         (
    #             ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    #             (F.col("status_her2") == "NEGATIVO") &
    #             (F.col("ki67_percentual") < 14)
    #         ),
    #         "Luminal A"
    #     ).when(
    #         (
    #             ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    #             (F.col("status_her2") == "NEGATIVO") &
    #             (F.col("ki67_percentual") >= 14)
    #         ),
    #         "Luminal B"
    #     ).when(
    #         (
    #             ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    #             (F.col("status_her2") == "POSITIVO")
    #         ),
    #         "Luminal com HER2 Positivo"
    #     ).when(
    #         (
    #             (F.col("receptor_estrogeno") == "NEGATIVO") &
    #             (F.col("receptor_progesterona") == "NEGATIVO") &
    #             (F.col("status_her2") == "POSITIVO")
    #         ),
    #         "HER-2 Superexpresso"
    #     ).when(
    #         (
    #             (F.col("receptor_estrogeno") == "NEGATIVO") &
    #             (F.col("receptor_progesterona") == "NEGATIVO") &
    #             (F.col("status_her2") == "NEGATIVO")
    #         ),
    #         "Triplo Negativo"
    #     ).otherwise("INDEFINIDO")
    # )

    # Base histórica
    #fs = FeatureStoreClient()
    #fs.create_table(
    #    name="refined.saude_preventiva.pardini_laudos_mamo_imunohistoquimico",
    #    primary_keys=["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"],
    #    schema=df_final.schema,
    #    description="Features extraídas de laudos de mamografia. Siglas: IH-NEO e IHMAMA"
    #)

    # Append em prd
    #num_linhas = df_final.count()
    #fs = FeatureStoreClient()
    #if num_linhas > 0:
    #    print(f"Há {num_linhas} registros para inserir — executando gravação…")
    #    primary_keys = ["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"]
    #    ###### Apenas para testes ##############
    #    df_final = df_final.dropna()
    #    df_final = df_final.dropDuplicates(primary_keys)
    #    ########################################
    #    fs.write_table(
    #        name="refined.saude_preventiva.pardini_laudos_mamo_imunohistoquimico",
    #        df=df_final,
    #        mode="merge",
    #    )
    #else:
    #    print("Nenhum registro encontrado; nada a fazer.")

    df_metrics = pd.DataFrame()
    df_metrics["laudos"] = laudos
    df_metrics["resultados"] = lista_dicts

    df_metrics["resultados"] = df_metrics["resultados"].apply(parse_json_string)

    lista_laudos2 = df_metrics["laudos"].tolist()
    lista_modelo2 = df_metrics["resultados"].tolist()

    lista_pseudo_gold2 = []
    lista_comparacoes2 = []

    for laudo_txt, json_mod in zip(lista_laudos2, lista_modelo2):
        json_heu, comp = avalia_extracao_sem_ground_truth_novo(laudo_txt, json_mod)
        lista_pseudo_gold2.append(json_heu)
        lista_comparacoes2.append(comp)

    df_metrics = pd.DataFrame()
    df_metrics["laudos"] = laudos
    df_metrics["resultados"] = lista_comparacoes2
    resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
    df_metrics = pd.concat(
            [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
            axis=1
        )
    
    # json_metricas = agrega_resultados_dinamico(lista_comparacoes2)

    # mlflow.set_experiment("/Users/aureliano.paiva@grupofleury.com.br/imunohistoquimico_pardini_metricas")

    # threshold = 0.8

    # with mlflow.start_run(run_name="Extracao_Laudos_Run_Threshold"):
    #     for campo, stats in json_metricas.items():
    #         taxa = stats["taxa_acerto"]
            
    #         # Registrar a taxa de acerto
    #         mlflow.log_metric(f"{campo}_taxa_acerto", taxa)
            
    #         # Registrar flag de aprovação no threshold
    #         passou_flag = 1 if taxa >= threshold else 0
    #         mlflow.log_metric(f"{campo}_passou_threshold", passou_flag)
            
    #         # Opcional: registrar acertos e total
    #         mlflow.log_metric(f"{campo}_acertos", stats["acertos"])
    #         mlflow.log_metric(f"{campo}_total", stats["total"])
        
    #     run_id = mlflow.active_run().info.run_id
    #     print(f"Run registrada: {run_id}")

## Contagem de Registros Processados

Esta célula executa uma operação simples mas importante: verificar a quantidade de registros no DataFrame final após todo o processamento.

```python
df_final_classif.count()
```

Esta operação aciona uma ação no DataFrame Spark e retorna um número inteiro representando a quantidade total de registros que foram processados com sucesso pelo pipeline completo.

Os resultados desta contagem são úteis para:

1. **Validação de Processamento**: Confirmar que o processamento foi bem-sucedido e que foram gerados resultados
2. **Controle de Qualidade**: Verificar se o número de registros de saída corresponde ao número de registros de entrada (df_spk.count())
3. **Monitoramento**: Obter uma métrica básica de volume para registro e acompanhamento

Esta verificação simples é parte importante do processo de validação dos resultados, garantindo que o pipeline de processamento está produzindo a quantidade esperada de registros.

In [ ]:
df_final_classif.count()

## Persistência dos Resultados em Tabela Delta

Esta célula final implementa o armazenamento persistente dos resultados processados em uma tabela Delta no catálogo Databricks. O código é estruturado para garantir robustez no processo de persistência, incluindo tratamento de erros e notificações.

### Componentes Principais:

1. **Importações e Configuração**:
   ```python
   import traceback
   from octoops import Sentinel  # Framework de monitoramento interno
   from delta.tables import DeltaTable  # API para manipulação de tabelas Delta
   ```
   - Importa bibliotecas para tratamento de exceções e manipulação de tabelas Delta
   - Configura variáveis de ambiente como `WEBHOOK_DS_AI_BUSINESS_STG`

2. **Definição do Caminho de Saída**:
   ```python
   OUTPUT_DATA_PATH = "refined.saude_preventiva.pardini_laudos_mama_imunohistoquimico"
   ```
   - Define o caminho completo para a tabela Delta de destino no formato `database.schema.table`

3. **Função de Inserção de Dados**:
   ```python
   def insert_data(df_spk, output_data_path):
       # Verifica se a tabela já existe
       if not DeltaTable.isDeltaTable(spark, output_data_path):
           # Cria a tabela se não existir
           df_spk.write.format("delta").saveAsTable(output_data_path)
       else:
           # Se existir, faz merge (upsert)
           delta_table = DeltaTable.forPath(spark, output_data_path)
           (delta_table.alias("target")
            .merge(...)
            .whenMatchedUpdateAll()  # Atualiza todos os campos se o registro já existir
            .whenNotMatchedInsertAll()  # Insere se o registro for novo
            .execute())
   ```
   - Implementa lógica para criar a tabela ou fazer merge em tabela existente
   - Usa chaves de identificação `ficha`, `id_exame`, `id_marca` e `sequencial` para identificar registros únicos

4. **Bloco de Execução com Tratamento de Exceções**:
   ```python
   try:
       if (df_final_classif.count() > 0):
           # Executa persistência de dados
           insert_data(df_final_classif, OUTPUT_DATA_PATH)
           print('Total de registros salvos na tabela:', df_final_classif.count())
       else:
           # Gera alerta se não há dados para processar
           error_message = "Pardini Imunuhistoquimico - Não há laudos para extração."
           sentinela_ds_ai_business = Sentinel(...)
           sentinela_ds_ai_business.alerta_sentinela(...)
   except Exception as e:
       # Captura e imprime erros, então reenvie a exceção
       traceback.print_exc()
       raise e
   ```
   - Verifica se há registros para persistir antes de tentar salvar
   - Envia notificações via Sentinel se não houver registros para processar
   - Captura e registra exceções, garantindo visibilidade de qualquer problema

Esta etapa final é crucial pois garante que os resultados do processamento sejam armazenados de forma persistente, permitindo acesso posterior para consultas, análises e integração com outros sistemas de saúde. O uso da tecnologia Delta Lake oferece transações ACID, garantindo integridade dos dados mesmo em casos de falhas durante a escrita.

In [ ]:
import traceback
from octoops import Sentinel
from delta.tables import DeltaTable

WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

# Iniciar sessão Spark
spark = SparkSession.builder.appName("DfPandasparaSpark").getOrCreate()

#OUTPUT_DATA_PATH = dbutils.widgets.get("OUTPUT_DATA_PATH")
OUTPUT_DATA_PATH = "refined.saude_preventiva.pardini_laudos_mama_imunohistoquimico"

# função para salvar dados na tabela
def insert_data(df_spk, output_data_path):

    # Cria a tabela Delta se não existir
    if not DeltaTable.isDeltaTable(spark, output_data_path):
        df_spk.write.format("delta").saveAsTable(output_data_path)
    else:
        # Carrega a tabela Delta existente
        delta_table = DeltaTable.forPath(spark, output_data_path)

        # Faz o merge (upsert)
        (delta_table.alias("target")
        .merge(
            df_spk.alias("source"),
            "target.ficha = source.ficha AND target.id_exame = source.id_exame AND target.id_marca = source.id_marca AND target.sequencial = source.sequencial"
        )
        .whenMatchedUpdateAll() #atualiza todos os campos se o ID já existir
        .whenNotMatchedInsertAll() #insere se o ID não existir
        .execute())

# salvar dados na tabela
try:
    # 1/0
    if (df_final_classif.count() > 0):        


        # Inserir tabela catalog
        insert_data(df_final_classif, OUTPUT_DATA_PATH)
        print('Total de registros salvos na tabela:', df_final_classif.count())
       

    else: 
        error_message = traceback.format_exc()
        error_message = "Pardini Imunuhistoquimico - Não há laudos para extração."
        sentinela_ds_ai_business = Sentinel(
            project_name='Monitor_Linhas_Cuidado_Mama',
            env_type=WEBHOOK_DS_AI_BUSINESS_STG,
            task_title='Pardini Mama Imunuhistoquimico'
        )

        sentinela_ds_ai_business.alerta_sentinela(
            categoria='Alerta', 
            mensagem=error_message,
            job_id_descritivo='4_Pardini_mama_Imunuhistoquimico'
        )
except Exception as e:
    traceback.print_exc()
    raise e    

## Exportação para Excel (Comentada)

Esta célula final contém código comentado que poderia ser usado para exportar os resultados processados para um arquivo Excel. Embora não esteja ativa, ela permanece como referência para casos em que seja necessária a exportação dos dados para análise externa.

```python
#%pip install openpyxl
#df_pandas = df_final.toPandas()
#df_pandas.to_excel("pardini_laudos_mamo_imunohistoquimico.xlsx", index=False)
```

O código possui três componentes principais:

1. **Instalação de Biblioteca**: `%pip install openpyxl` instalaria a biblioteca necessária para manipulação de arquivos Excel
2. **Conversão para Pandas**: `df_final.toPandas()` converteria o DataFrame Spark em um DataFrame Pandas
3. **Exportação para Excel**: `to_excel()` salvaria os dados em um arquivo Excel nomeado "pardini_laudos_mamo_imunohistoquimico.xlsx" sem incluir o índice

Este recurso poderia ser útil para:
- Análises locais dos dados processados
- Compartilhamento de resultados com equipes que não têm acesso ao ambiente Databricks
- Visualizações e formatações específicas usando Excel

No entanto, em um ambiente de produção com grande volume de dados, a abordagem preferida é usar o armazenamento em tabelas Delta, como implementado na célula anterior.

In [ ]:
#%pip install openpyxl
#df_pandas = df_final.toPandas()
#df_pandas.to_excel("pardini_laudos_mamo_imunohistoquimico.xlsx", index=False)